In [ ]:
# 필요 시 주석을 해제하고 아래 라이브러리들을 설치
# !pip install pystan==2.19.1.1
# !pip3 install fbprophet
# !pip install plotly

In [1]:
# 라이브러리 임포트
import os
import time
import json
import datetime
import numpy as np
import pandas as pd
from fbprophet import Prophet

In [2]:
%%time
# 데이터 로드
train = json.load(open('./data/train/train.json', 'r', encoding='utf8')) 
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 

CPU times: user 3.64 s, sys: 551 ms, total: 4.2 s
Wall time: 4.2 s


In [3]:
%%time
# 사업장별 dataframe 제작
df_dic = {}
plc_lst = list(train.keys())
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

for i in plc_lst:
    plc_df = pd.DataFrame(columns={'ds', 'pH', 'COD', 'SS', 'N', 'P', 'T'})
    plc_df = plc_df[['ds', 'pH', 'COD', 'SS', 'N', 'P', 'T']]
    
    date_lst = list(train[i].keys())
    plc_df.loc[:, 'ds'] = date_lst

    for index, j in enumerate(date_lst):
        for k in fct_lst:
            try:
                plc_df.loc[index, k] = train[i][j][k]
            except:
                pass
                    
    df_dic[i] = plc_df

CPU times: user 4min 30s, sys: 19.6 ms, total: 4min 30s
Wall time: 4min 30s


In [27]:
df_dic['삼성SDI(울산)'].dtypes

,ds,pH,COD,SS,N,P,T
0,20100101,1.237804,1.104691,0.166969,0.359743,-0.365124,-0.152997
1,20100102,0.652694,1.790647,0.166969,0.623086,-0.261693,-0.152997
2,20100103,0.067584,1.733484,0.962297,0.908991,-0.227216,-0.152997
3,20100104,0.652694,1.733484,0.697188,1.541828,-0.330647,-0.152997
4,20100105,1.237804,1.733484,0.166969,2.29783,-0.322028,-0.152997
...,...,...,...,...,...,...,...
2826,20180127,-1.102636,-0.667363,-0.363249,NaN,NaN,-0.152997
2827,20180128,-1.102636,-0.6102,-0.628359,NaN,NaN,-0.152997
2828,20180129,-1.102636,-0.6102,-0.628359,NaN,NaN,-0.152997
2829,20180130,-1.102636,-0.553037,-0.628359,NaN,NaN,-0.152997


In [ ]:
plc_lst[:5]

In [7]:
df_dic[plc_lst[0]]

,ds,pH,COD,SS,N,P,T
0,20100101,0.483936,1.273783,1.364365,1.268182,1.148905,0.115951
1,20100102,0.483936,1.150021,1.280543,1.392281,0.544579,0.115951
2,20100103,0.877648,0.94375,1.029079,1.363081,0.678874,0.115951
3,20100104,0.877648,0.861242,1.615829,1.903861,1.23549,0.115951
4,20100105,0.877648,0.654972,1.112901,2.235863,1.403358,0.115951
...,...,...,...,...,...,...,...
2815,20180127,2.058786,0.366194,1.448186,2.200823,-1.024548,-0.136248
2816,20180128,2.058786,0.32494,1.615829,1.963429,-0.983906,-0.136248
2817,20180129,2.058786,0.366194,1.532008,1.933645,-1.010412,-0.136248
2818,20180130,1.665074,1.108767,1.532008,2.450481,-0.992741,-0.136248


In [3]:
import pickle

In [10]:
%%time
#pickle.dump(df_dic , open('./mydata/df_dic.plk','wb'))
#pickle.dump(plc_lst , open('./mydata/plc_lst.plk','wb'))

In [4]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

CPU times: user 874 ms, sys: 125 ms, total: 999 ms
Wall time: 999 ms


In [5]:
# Prophet 예측을 위한 dataframe 포맷 제작
future = pd.DataFrame([str(x)[:10] for x in list(pd.date_range(start='2018-02-01', end='2019-12-31', inclusive="both"))], columns=['ds'])
future['y'] = np.nan
#future

In [ ]:
# 추론
start_time = time.time()

for i in plc_lst:
    plc_df = df_dic[i]
    
    for j in fct_lst: # 피쳐 리스트
        fct_df = plc_df[['ds', j]]   # 날짜 - 피쳐1 조합임.
        fct_df = fct_df.rename(columns={j:'y'}) # 피쳐1을 y로 바꾼다.
        model = Prophet()
        model.fit(fct_df)
        forecast = model.predict(future)
        forecast['ds'] = forecast['ds'].apply(lambda x : str(x).split(' ')[0].split('-')[0]+str(x).split(' ')[0].split('-')[1]+str(x).split(' ')[0].split('-')[2])
        forecast = forecast[['ds', 'yhat']]
        
        for index, k in enumerate(list(forecast['ds'])):
            submit[i][forecast.loc[index, 'ds']][j] = np.float(forecast[forecast['ds']==k]['yhat'])
        
end_time = time.time()
print('소요시간 :', (end_time - start_time)/60, 'minutes')

In [23]:
# 추론 파일 저장
with open('./sub/baseline.json', 'w', encoding='utf8') as f:
    json.dump(submit, f, ensure_ascii=False)